## 0. Инструменты

Импортнем все необходимое.

In [230]:
import os
import re
import json
import random
import numpy as np
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from pymystem3 import Mystem

## 1. Данные

В качестве корпуса я взяла корпус субтитров к 9 сезону реалити-шоу "RuPaul's Drag Race". Он содержит субтитры к 13 эпизодам.

In [195]:
path = '/home/deltamachine/Desktop/projects/rupaul-subtitles-analysis/subtitles/s9/'

files = os.listdir(path)
corpus = []

for file in files:
    with open(path + file, 'r', encoding="utf-8") as file:
        corpus.append(file.read())

## 2. Препроцессинг
Поступим так:

1) Переведем текст в нижний регистр

2) Выбросим все теги, нечитаемые символы, специфические символы типа ♪, а также отметки времени (корпус - это файлы в формате .srt, т.е. реальные непочищенные субтитры), а также пунктуацию и лишние пробелы

3) Токенизируем и лемматизируем текст.

4) Отсеем стоп-слова, причем не только те, которые нам предлагает NLTK, но и локальные - для этого будем выкидывать 15 самых частотных слов в эпизоде, а также составлять общий список "локальных" стоп-слов.

In [198]:
punct = punctuation + '«»—…“”*№–'
all_local_stops = set()
stops = set()

In [218]:
def normalize_text(text, lang):
    global all_local_stops 
    global stops
    
    if lang == 'en':
        stops = set(stopwords.words('english'))
        lemmatizer = WordNetLemmatizer()
    elif lang == 'ru':
        stops = set(stopwords.words('russian'))
        lemmatizer = Mystem()
    
    text = text.lower()
    text = re.sub('[0-9]+:[0-9]+.*?\n', ' ', text)
    text = re.sub('<.*?>', ' ', text)
    text = re.sub('\ufeff1', ' ', text)
    text = re.sub('\n[0-9]*', ' ', text)
    text = re.sub('♪', ' ', text)
    text = re.sub('\s{2,}', ' ', text)

    text = [word for word in word_tokenize(text) if word not in punct and word not in stops]  
    
    if lang == 'en': 
        text = [lemmatizer.lemmatize(word) for word in text]
    elif lang == 'ru':
        text = [lemmatizer.lemmatize(word)[0] for word in text]
    
    local_stops = [elem[0] for elem in Counter(text).most_common(15)]
    all_local_stops |= set(local_stops)
    
    text = ' '.join([word for word in text if word not in local_stops])

    return text

In [202]:
normalized_corpus = []

for text in corpus:
    normalized_text = normalize_text(text, 'en')
    normalized_corpus.append(normalized_text)

## 4. Извлечение ключевых слов

Конкретно для этого корпуса большой смысл имеет TF-IDF - надо ведь посмотреть, чем конкретно эпизоды отличаются друг от друга.

Применим его и выведем по 5 ключевых слов для каждого эпизода.

In [203]:
stops |= all_local_stops

In [204]:
def keywords_extraction(corpus, stops):
    vectorizer = TfidfVectorizer(stop_words=stops)
    matrix = vectorizer.fit_transform(corpus)

    id2word = {i:word for i,word in enumerate(vectorizer.get_feature_names())} 

    for text_row in range(matrix.shape[0]):
        row_data = matrix.getrow(text_row) 
        all_words = row_data.toarray().argsort() 
        top_words = all_words[0,:-6:-1] 
        print([id2word[w] for w in top_words])

In [205]:
keywords_extraction(normalized_corpus, stops)

['coo', 'shack', 'jaymes', 'floozy', '52']
['dah', 'gah', 'kah', 'category', 'four']
['pageant', 'vega', 'see', 'new', 'video']
['charlie', 'naya', 'eureka', 'two', 'show']
['duncan', 'crew', 'sarge', 'heel', 'sister']
['monna', 'dartin', 'mom', 'prom', 'action']
['greedy', 'club', 'okay', 'pilot', 'kid']
['roast', 'burning', 'farrah', 'okay', 'time']
['toot', 'fan', 'boot', 'valentina', 'lip']
['inspired', 'aesthetic', 'kimora', 'city', 'season']
['madonna', 'jasmine', 'marlene', 'snatch', 'dietrich']
['macho', 'rainbow', 'unicorn', 'village', 'rhythmic']
['starfish', 'sidekick', 'aquapussy', 'fire', 'hero']


Результаты достаточно осмысленные - по ключевым словам довольно легко угадать эпизод.

Например, массив ['duncan', 'crew', 'sarge', 'heel', 'sister'] описывает эпизод , в котором принимали участие члены съемочной группы ("сrew") - в частности, люди, которых звали Duncan и Sarge.

А массив ['madonna', 'jasmine', 'marlene', 'snatch', 'dietrich'] описывает эпизод с челленджем под названием "Snatch Game", в котором неоднократно упоминались Марлен Дитрих и Джасмин Мастерс.

## 5. Обработка другого датасета

Возьмем корпус текстов Russia Today и посмотрим, сколько в нем текстов.

In [209]:
test_corpus = []

with open('/home/deltamachine/Downloads/russia_today_0.jsonlines', 'r') as file:
    for line in file.readlines():
        test_corpus.append(json.loads(line))

In [228]:
len(test_corpus)

999

Я уже не укладываюсь в дедлайн и процессинг всех текстов займет очень много времени - поэтому я просто выберу b обработаю 13 случайных, по аналогии с 13 файлами в исходном корпусе.

In [235]:
all_local_stops = set()

In [236]:
normalized_test_corpus = []
indexes = random.sample(range(0, 1000), 13)

for i in indexes:
    normalized_test_text = normalize_text(test_corpus[i]['content'], 'ru')
    normalized_test_corpus.append(normalized_test_text)

In [242]:
indexes[0]

774

In [248]:
true_keywords = []

for i in indexes:
    true_keywords.append(test_corpus[i]['keywords'][:5])

[['биатлон',
  'кёрлинг',
  'лыжный спорт',
  'ои-2018 биатлон',
  'ои-2018 конькобежный спорт'],
 ['алеппо', 'боевики', 'в мире', 'сирия', 'видео'],
 ['зарплата', 'здоровье', 'медицина', 'новости россии', 'политика'],
 ['в мире', 'вашингтон', 'дональд трамп', 'кндр', 'ким чен ын'],
 ['австралия', 'в мире', 'джеймс мэттис', 'дональд трамп', 'кндр'],
 ['бокс', 'в мире', 'россия', 'скандал', 'спорт'],
 ['евразийский союз', 'еврокомиссия', 'конституция', 'национализм', 'польша'],
 ['бокс', 'россияне', 'сша', 'сергей ковалёв', 'спорт'],
 ['в мире',
  'дональд трамп',
  'исламское государство',
  'кндр',
  'конфликт сша и кндр'],
 ['автомобиль', 'в россии', 'гибдд', 'деньги', 'новости россии'],
 ['интервью', 'магнус карлсен', 'сергей карякин', 'спорт', 'шахматы'],
 ['безопасность', 'евровидение', 'конкурс', 'крым', 'песня'],
 ['видео', 'геноцид армян', 'кино', 'культура']]

In [251]:
stops |= all_local_stops

In [250]:
keywords_extraction(normalized_test_corpus, stops)

['медаль', 'шанс', 'альпензия', 'шорт', 'квалификация']
['join', 'ярошевский', 'интеграция', 'информированность', 'информация']
['медик', 'волновать', 'персонал', 'хромов', 'лекарство']
['асмолов', 'jwplayer', 'ga', 'players', 'трамп']
['корейский', 'недовольство', 'путем', 'адмирал', 'белый']
['сборная', 'бойкотировать', 'кремлев', 'соревнование', 'украинец']
['брюссель', 'ек', 'качиньский', 'еврокомиссия', 'справедливость']
['раунд', 'оппонент', 'боксер', 'отдавать', 'бокс']
['нескромный', 'корея', 'пхеньян', 'рейган', 'влияние']
['рубль', 'транспортный', 'совершать', 'средство', 'предусматривать']
['ваш', 'карлсен', 'карякин', 'ребенок', 'игра']
['въезд', 'приезжать', 'проект', 'певица', 'участница']
['актриса', 'оскар', 'национальность', 'звезда', 'французский']


...Видно, что ключевые слова совпадают плохо. Во многом, потому, что в "оригинальных" ключевых словах много словосочетаний. И потому, что алгоритм выдает 5 самых "специфичных" слов, а в тестовом корпусе у нас нет возможности отобрать 5 самых специфичных из уже определенных ключевых слов, поэтому мы взяли 5 первых (довольно тупо, но что уж поделать).

А еще, возможно, для этого корпуса не имело смысла удалять самые частотные слова после исключения стоп-слов (а для исходного корпуса - однозначно имело). Мораль: подходи к каждым данным индивидуально!

## 6. Оценка результатов

По причинам, указанным выше, прогонять функцию оценки не особо имеет смысл, но я ее все-таки напишу.

In [ ]:
def evaluate(true_keys, predicted_keys):
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_keys)):
        true_keys = set(true_keys[i])
        predicted_kw = set(predicted_keys[i])
        
        tp = len(true_keys & predicted_keys)
        union = len(true_keys | predicted_keys)
        fp = len(predicted_keys - true_keys)
        fn = len(true_keys - predicted_keys)
        
        if (tp + fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp + fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
            
        if (prec + rec) == 0:
            f1 = 0
        else:
            f1 = (2 * (prec * rec)) / (prec + rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
        
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))